In [1]:
import torch
from torch.nn import Identity
import geoopt
from geoopt.optim import RiemannianSGD, RiemannianAdam
import matplotlib.pyplot as plt

import numpy as np
from torch.func import grad

from tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
from torchvision.transforms.functional import rotate
from scipy.linalg import lstsq, null_space

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from torch.optim import Adam

In [2]:
# Get cpu or gpu device for training.
device = "cuda"
print(f"Using {device} device")

Using cuda device


In [3]:
def G(d): # questa è  rot_mat
    mat = np.zeros(d**4).reshape(d**2,d**2)
    for j in range(d):
        for i in range(d):
            k = (i+1)*d-j
            h = i+(j)*d+1

            mat[h-1][k-1] = 1
    return mat

def W(output_dim, d = 4):
    kernel = null_space((np.identity(d*d) -  G(d)).T).T
    W = []
    for j in range(output_dim):
        vec = np.zeros(d**2)
        for k in kernel:
            vec = np.random.random()*k + vec
        W.append(list(vec))
    W = np.array(W)/len(kernel)
    return np.array(W)

In [4]:
# load dataset
(trainX, trainy), (testX, testy) = mnist.load_data()

trainX = trainX[:10]
trainy = trainy[:10]

trainy = torch.Tensor(trainy).to(device)
trainX = trainX / 256
testX = testX / 256

trainX = torch.Tensor(trainX).to(device)
print(trainX.shape)
trainX_90 = rotate(img = trainX, angle = 90)

# summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

torch.Size([10, 28, 28])
Train: X=torch.Size([10, 28, 28]), y=torch.Size([10])
Test: X=(10000, 28, 28), y=(10000,)


In [5]:
def flatten(weights_list):
    flatten_weights = []
    for weight in weights_list:
        flatten_weights.append(weight.flatten())
        
    return torch.concatenate(flatten_weights)

def deflatten(weights_flat, shapes):
    weigths_list = []
    shape_counter = 0
    for shape in shapes:
        weight = torch.Tensor(weights_flat[shape_counter: shape_counter + np.prod(shape)])
        shape_counter = shape_counter + np.prod(shape)
        
        weigths_list.append(weight.reshape(shape))
            
    return weigths_list

# Funzione che definisce la varietà: sfera unitaria centrata in 0
def g(thetas):
    
    W1, b1, W2, b2, W3, b3 = deflatten(thetas, model.get_shapes())
    I_minus_G = torch.from_numpy(np.identity(28*28) - G(28)).to(device).float()
    norm = (W1.requires_grad_(True) @ I_minus_G).norm()**2
    
        
#     # Zero grad
#     for param in model.parameters():
#         param.grad = torch.zeros(param.shape).to(device)
        
    # Compute gradient
    norm.backward()
    
    return norm # sfera unitaria


def dg(thetas):
    _ = g(thetas)
    def_grad = [param.grad for param in model.parameters()]
    
    
    return flatten(def_grad)

# # Gradiente di f
# def dg(thetas):
#     gradient = grad(g)(thetas)
#     return gradient #+ 1e-5*(gradient.norm()==0)


# Classe LevelSetManifold già implementata sopra
class LevelSetManifold(geoopt.manifolds.Manifold):
    
    ndim = 1
    name = "Caste"
    
    def __init__(self, f, df, lr_proj = 1):
        super().__init__()
        self.f = f
        self.df = df
        self.lr_proj = lr_proj

    def _check_point_on_manifold(self, x, atol=1e-7, rtol=1e-7):
        return torch.abs(self.f(x)) < atol

    def _check_vector_on_tangent(self, x, u, atol=1e-7, rtol=1e-7):
        grad_f = self.df(x)
        return torch.abs(u @ grad_f).sum() < atol
    
    def projx(self,x):
        if self._check_point_on_manifold(x):
            return x
        for r in range(100):
            x = self.single_projx(x)
            if r%50==0:
                print("g: ", self.f(x))
                print("dg: ", self.df(x).norm())
            if r == 99:
                print(f"Retraction applied {r + 1} times")
            if self._check_point_on_manifold(x):
                print(f"Retraction applied {r + 1} times")
                break
        return x
    
    def single_projx(self, x):
        """
        Retraction
        """
        grad_f = self.df(x)
        f_val = self.f(x)
        return x - 1*self.lr_proj*(f_val / grad_f.norm()**2 * grad_f)

    def proju(self, x, u):
        """
        Projected gradient
        """
        print("Sono stato chiamato")
        grad_f = self.df(x)
        return u - 1*(u @ grad_f) / grad_f.norm()**2 * grad_f

    def inner(self, x, u, v=None):
        if v is None:
            v = u
        return (u * v).sum()

    def expmap(self, x, u):
        return self.retr(x, u)

    def egrad2rgrad(self, x, u):
        return self.proju(x, u)

    def retr(self, x, u):
        x_new = x + u
        return self.projx(x_new)

In [6]:
from torch import nn


class InvariantNN(nn.Module):
    def __init__(self, input_dim, hidden, output):
        super(InvariantNN, self).__init__()
        
#         self.softmax = nn.Softmax()
        
        self.input_dim = input_dim
        self.hidden = hidden
        self.output = output
    
        self.flatten = nn.Flatten(start_dim=1)
        self.l1 = nn.Linear(self.input_dim, self.hidden)
        self.l2 = nn.Linear(self.hidden, self.hidden)
        self.l3 = nn.Linear(self.hidden, self.output)
        
        self.l1.weight.data = torch.Tensor(W(self.hidden,d=int(self.input_dim**0.5))).requires_grad_(True)
        
        self.leakyrelu = nn.LeakyReLU()
        self.tanh = nn.Tanh()
        
        self.acc_loss = nn.CrossEntropyLoss()
        
    def get_shapes(self):
        shapes = [(self.hidden, self.input_dim), 
                  (self.hidden,1),
                  (self.hidden, self.hidden),
                  (self.hidden,1),
                  (self.output,self.hidden),
                  (self.output,1)
                 ]
        return shapes
        

    def forward(self, x):
        x = self.flatten(x)
        x = self.l1(x)
        x = self.tanh(x)
        x = self.l2(x)    
        x = self.tanh(x)
        x = self.l3(x)

        return  x
    
model = InvariantNN(28*28, 64, 10).to(device)

In [7]:
# Creiamo la varietà
manifold = LevelSetManifold(g, dg, 1)
# Test del parametro manifold

theta = flatten(model.parameters())
theta = geoopt.ManifoldParameter(theta , manifold=manifold)

# print("Prima: ", "g: ", g(theta).item(), "dg: ", dg(theta))


# # Verifica che il parametro iniziale appartenga alla varietà
# assert manifold._check_point_on_manifold(theta.data), "Il punto iniziale non è sulla varietà"

# Definiamo una loss function: minimizziamo la norma quadrata
def loss_fn(trainX):
    return torch.nn.CrossEntropyLoss()(model(trainX), trainy.long())

# Ottimizzatore Riemanniano
optimizer = RiemannianSGD([theta], lr=0.01)

# Ciclo di ottimizzazione
g_during_train = []
loss_history = []
print("############## Training phase ################")
for epoch in range(300):
    optimizer.zero_grad()
    loss = loss_fn(trainX)
    loss.backward()
    
    print("\n", sum(theta))
    optimizer.step()
    print(sum(theta), "\n")
    
    if epoch%10 == 0:
        print("Epoch:",  epoch + 1, 
              "Loss: ", loss.item(),
              "g(theta): ", round(float(g(theta).data),8),
              "dg: ", dg(theta).data,
              #"Sum of grad: ", sum(sum(model.l3.weight.grad)),
              "Output: ", sum(model(trainX)[0]).item(), sum(model(trainX)[1]).item()
             )
    loss_history.append(loss.item())
    g_during_train.append(g(theta).data)
# plt.plot(g_during_train)
plt.plot(loss_history)
print("\n\n")

# Risultato finale
print("Punto finale:", theta)
print("Appartiene alla varietà?", manifold._check_point_on_manifold(theta))

############## Training phase ################

 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>) 

Epoch: 1 Loss:  2.320369243621826 g(theta):  0.0 dg:  tensor([ 0.0000,  0.0000,  0.0000,  ...,  0.1075,  0.1106, -0.0098],
       device='cuda:0') Output:  0.07906430214643478 0.07626278698444366

 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>) 


 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>) 


 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>) 


 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>) 


 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>) 


 tensor(6.1038, device='cuda:0', grad_fn=<AddBackward0>)
ten

KeyboardInterrupt: 

In [ ]:
model(torch.rand((1,28*28)).to(device))

In [ ]:
trainX_180 = rotate(img = trainX, angle = 180)
trainX_270 = rotate(img = trainX, angle = 270)

print(model(trainX)[1])
print(model(trainX_90)[1])
print(model(trainX_180)[1])
print(model(trainX_270)[1])

print((model(trainX)[3] - model(trainX_180)[3]).norm()**2)

In [ ]:
W1, b1, W2, b2, W3, b3 = deflatten(theta, model.get_shapes())
I_minus_G = torch.from_numpy(np.identity(28*28) - G(28)).to(device).float()
norm = (W1.requires_grad_(True) @ I_minus_G).norm()**2

In [ ]:
plt.imshow(G(28))

In [ ]:
plt.imshow(I_minus_G.cpu())

In [ ]:
plt.imshow(W1.cpu().detach().numpy().T @ I_minus_G.cpu().detach().numpy())

In [ ]:
W_model = model.l1.weight.data
plt.imshow(W_model.cpu().detach().numpy() @ I_minus_G.cpu().detach().numpy())

In [ ]:
W1.T @ trainX[0].flatten()  - W1.T @ trainX_90[0].flatten()

In [ ]:
plt.imshow(trainX[0].cpu())

In [ ]:
model.parameters() = deflatten(thetas, model.get_shapes())
NN1 = model(trainX)
NN2 = model(trainX_90)
Delta_NN = NN1 - NN2
norm = Delta_NN.norm()**2
print(norm)

In [ ]:
model.parameters()